In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from random import randint
from time import sleep

from datetime import date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials

import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Desktop/UpWork/gs-credentials.json', scope) 

# BetterWood

### Products Links

In [ ]:
%%time
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
data = g2d.download(gfile=spreadsheet_key, wks_name = 'BetterWood', 
                    credentials=credentials, col_names=True, row_names=True)
data.shape

### Images

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless=False
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [ ]:
images_df = pd.DataFrame({'Images':np.nan, 'URL':data['URL'].unique()}, index=range(data['URL'].nunique()))
images_df.shape

In [ ]:
for idx in images_df.index:
    
    print('\n#', idx)

    url = images_df['URL'].loc[idx]
    browser.get(url)
    sleep(1)
    
    img_urls = []
    path = '//*[contains(@class, "woocommerce-product-gallery")]/a'
    els = browser.find_elements_by_xpath(path)
    for el in els:
        url = el.get_attribute('href')
        if url not in img_urls:
            img_urls.append(url)
    
    print('; '.join(img_urls))
    images_df['Images'].loc[idx]= '; '.join(img_urls)

In [ ]:
images_df['Images'] = images_df['Images'].apply(lambda x: np.nan if x=='' else x)
data = pd.merge(data, images_df, on='URL', how='left')
data.head(2)

### Storing to GS

In [ ]:
gc = gspread.authorize(credentials)
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
def to_gs(data, sheet):
    d2g.upload(data.fillna('-'),
               spreadsheet_key,
               sheet,
               credentials=credentials,
               col_names=True,
               row_names=False,
               start_cell = 'A1',
               clean=True)
to_gs(data, 'BetterWood Test')

# Obi

### Products Links

In [ ]:
%%time
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
data = g2d.download(gfile=spreadsheet_key, wks_name = 'OBI Clean Data', 
                    credentials=credentials, col_names=True, row_names=True)
data.shape

### Images

In [ ]:
images_df = pd.DataFrame({'Images':np.nan, 'URL':data['URL'].unique()}, index=range(data['URL'].nunique()))
images_df.shape

In [ ]:
%%time 
for idx in images_df.index[100:]:
    try:
        url = images_df['URL'].loc[idx]
        browser.get(url)
        sleep(0.5)

        # How many images in gallery
        path = '//*[contains(@class, "ads-previewslider__item slick-slide")]/a'
        els = browser.find_elements_by_xpath(path)

        # Click to expand the image
        path = '//*[@class="ads-slider__zoom-text"]'
        browser.find_elements_by_xpath(path)[0].click()
        sleep(0.3)

    except:
        continue
    
    
    img_urls = []  
    if len(els)>1:

        for _ in range(len(els)-1):
            # Sliding gallery
            try:
                path = '//*[contains(@class, "ads-slider__slick slick-initialized slick-slider")]/button'
                browser.find_elements_by_xpath(path)[1].click()
                sleep(0.7)

                # Get the image
                path = '//*[@class="pinch-zoom-container"]/img'
                els = browser.find_elements_by_xpath(path)
                for el in els:
                    img = el.get_attribute('src')
                    if img not in img_urls and 'blind.gif' not in img:
                        img_urls.append(img)
            except:
                pass
    else:
        try:
            # Get the image
            path = '//*[@class="pinch-zoom-container"]/img'
            els = browser.find_elements_by_xpath(path)
            for el in els:
                img = el.get_attribute('src')
                if img not in img_urls and 'blind.gif' not in img:
                    img_urls.append(img)
        except:
            continue


    print('\n#', idx, '; '.join(img_urls))
    images_df['Images'].loc[idx]= '; '.join(img_urls)

    if idx%50==0 and idx>0:
        sleep(randint(1,10))


In [ ]:
images_df.to_csv('OBI-Images.csv')

In [ ]:
images_df['Images'] = images_df['Images'].apply(lambda x: np.nan if x=='' else x)
data = pd.merge(data, images_df, on='URL', how='left')
data.head(2)

In [ ]:
to_gs(data, 'OBI Test')

# SCS

In [ ]:
%%time
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
data = g2d.download(gfile=spreadsheet_key, wks_name = 'SCS Clean Data', 
                    credentials=credentials, col_names=True, row_names=True)
data.shape

In [ ]:
images_df = pd.DataFrame({'Images':np.nan, 'URL':data['URL'].unique()}, index=range(data['URL'].nunique()))
images_df.shape

In [ ]:
url = images_df['URL'].loc[55]
browser.get(url)

In [ ]:
%%time
for idx in images_df.index:
    
    try:
        url = images_df['URL'].loc[idx]
        browser.get(url)
        sleep(1.5)
    
        # Click to expand the image
        path = '//*[@class="product-view__main-image product-image product-image-zoom"]'
        browser.find_elements_by_xpath(path)[0].click()
        sleep(1)

        # Get the image
        path = '//*[@class="mz-figure mz-hover-zoom mz-ready"]/img'
        img = browser.find_elements_by_xpath(path)[0].get_attribute('src')
        images_df['Images'].loc[idx]= img
        
        print('#',idx, url ,img)
      
    except:
        print('ERROR WITH IMAGE')
        continue

In [ ]:
images_df['Images'] = images_df['Images'].apply(lambda x: np.nan if x=='' else x)
data = pd.merge(data, images_df, on='URL', how='left')
data.head(2)

In [ ]:
to_gs(data, 'SCS Test')

# Toom

In [ ]:
%%time
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
data = g2d.download(gfile=spreadsheet_key, wks_name = 'Toom Clean Data', 
                    credentials=credentials, col_names=True, row_names=True)
data.shape

In [ ]:
images_df = pd.DataFrame({'Images':np.nan, 'URL':data['URL'].unique()}, index=range(data['URL'].nunique()))
images_df.shape

In [ ]:
%%time
for idx in images_df.index[10:]:
    
    try:
        url = images_df['URL'].loc[idx]
        browser.get(url)
        sleep(0.5)
    
        # Click to expand the image
        path = '//*[@class="a-icon a-icon--magnify-plus fullscreen-gallery__zoom-button-icon"]'
        browser.find_elements_by_xpath(path)[0].click()
        sleep(0.3)

        # Get the image
        path = '//*[@class="image-gallery-image"]'
        img = browser.find_elements_by_xpath(path)[0].get_attribute('src')
        images_df['Images'].loc[idx]= img
        
        print('#',idx, url ,img)
        if idx%50==0 and idx>0:
            sleep(randint(1,10))
      
    except:
        print('ERROR WITH IMAGE')
        continue

In [ ]:
images_df['Images'] = images_df['Images'].apply(lambda x: np.nan if x=='' else x)
data = pd.merge(data, images_df, on='URL', how='left')
data.head(2)